In [46]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import threshold_otsu
import os
from PIL import Image

In [81]:
test_directory = './data/43290879-American_Samoa/'
test_header = './data/43290879-American_Samoa/samoa-header.jpg'
out_folder = 'segdata/American_Samoa/blocks/'

In [84]:
# for root, dirs, files in os.walk(test_directory):
#     for filename in files:
#         if len(dirs)==0:
#             print(root+'/'+filename)

In [82]:
def block_segmenter(directory,header,outputfolder):
    head_img = cv2.imread(header)
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if len(dirs) == 0:
                fname = root + '/' + filename
                img = cv2.imread(fname)
                crop_img = edge_clip_rotate(img)
                temp_suc,seg_img = template_match(crop_img,head_img)
                if temp_suc:
                    cv2.imwrite(outputfolder+'seg-'+filename,seg_img)
                    
                else:
                    cv2.imwrite(outputfolder+'fail-'+filename,seg_img)
                    
               
            

In [16]:
def edge_clip_rotate(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    _,thresh = cv2.threshold(gray, np.mean(gray), 255, cv2.THRESH_BINARY_INV)
    edges = cv2.dilate(cv2.Canny(thresh,0,255), None)
    cnt = sorted(cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2], key = cv2.contourArea)[-1]
    mask = np.zeros(gray.shape[0:2],np.uint8)
    masked = cv2.drawContours(mask, [cnt],-1,255,-1)
    
    left = cnt[cnt[:,:,0].argmin()][0]
    right = cnt[cnt[:,:,0].argmax()][0]
    top = cnt[cnt[:,:,1].argmin()][0]
    bottom = cnt[cnt[:,:,1].argmax()][0]
    
    lb = (bottom[0]-left[0])/(bottom[1]-left[1])
    rb = (right[1]-bottom[1])/(bottom[0]-right[0])
    tl = (left[1]-top[1])/(top[0]-left[0])
    tr = (right[0]-top[0])/(right[1]-top[1])
    
    
    if right[1]<left[1]: #if right_y <left_y, that is, page is shifted cw relative to straight
        angle = sum([np.arctan(1/tl),np.arctan(1/rb)])*90/np.pi
    else:
        angle = sum([np.arctan(tr),np.arctan(lb)])*-90/np.pi
        multiplier = -1
    cropped_image = image[left[0]:right[0],top[1]:bottom[1]]
    (h,w) = cropped_image.shape[:2]
    center = (w/2,h/2)
    rotmat = cv2.getRotationMatrix2D(center,angle,1)
    croprot_image = cv2.warpAffine(cropped_image,rotmat,(w,h))
    return croprot_image

In [24]:
def template_match(cropped, header):
    if header.shape[1]>cropped.shape[1]:
        return False, cropped
    w,h = header.shape[::-1][1:3]
    res = cv2.matchTemplate(cropped,header,cv2.TM_CCOEFF)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0]+w,top_left[1]+h)
    segmented = cropped[bottom_right[1]:,top_left[0]:bottom_right[0],:]
    return True, segmented

In [85]:
#block_segmenter(test_directory,test_header,out_folder)

Pretty good- some duds, but quite a few came out very nicely. You might want to add a test to see if you need to do the crop/rotation part- ex: Alabama clearly doesn't need it, so putting it in there might be a notable loss by doing it. But, it's to be seen. If you just run Alabama and it works fine, why worry?